In [ ]:
import pandas as pd

path = '..'

In [ ]:
df = pd.read_csv(f'{path}/dataset/GetSBProfileInfoService/getOtlInfo/all.csv')

In [ ]:
tmp = pd.read_csv(f'{path}/dataset/GetSBProfileInfoService/getCsdoStatus/all.csv')

In [10]:
key=list(df.columns)

In [11]:
df = pd.merge(df, tmp, on=key, how = 'left')

In [34]:
tmp = df.groupby(['basYm', 'rprSexNm', 'rprAggrNm', 'estbYr', 'bizAreaNm', 'bizBzcCd', 'empeCntNm']).count().reset_index() # 이걸론 unique 불가능 -> 분석은 가능할 듯???

In [30]:
# 년도별 설립 확인 가능할 듯?????? -> 재무상태는 나중에 확인하고 일단 업종...
df[(df['rprSexNm']=='남성') & (df['rprAggrNm']=='30대') & (df['estbYr']==1972) & (df['bizAreaNm']=='서울특별시 서대문구')] # 3명 존재... 개인으로는 확인 불가능 같은년도에 3명....

,basYm,rprSexNm,rprAggrNm,estbYr,bizAreaNm,bizBzcCd,bizBzcCdNm,empeCntNm
791357,202208,남성,30대,1972,서울특별시 서대문구,47,소매업; 자동차 제외,1명 이상 5명 미만
1781357,202208,남성,30대,1972,서울특별시 서대문구,47,소매업; 자동차 제외,1명 이상 5명 미만
2781357,202208,남성,30대,1972,서울특별시 서대문구,47,소매업; 자동차 제외,1명 이상 5명 미만
3898400,202208,남성,30대,1972,서울특별시 서대문구,47,소매업; 자동차 제외,1명 이상 5명 미만


In [ ]:
tmp['bizBzcCdNm'].unique() # unique가 안됨... 합칠 수 없음

array([  4,   2,   8,   3,  12,  20,   6,  16,  32,  28,  24,  40,  36,
         9,  15,   5,  48,  44,  10,  64,  52, 104,  68,  60,  56,  84,
        76,  92,  21,  14,  22,  18,  72, 108,  80, 124,  88, 100, 116,
        17,  96, 152,  30, 204, 112, 156, 136, 128, 120,  34,  11, 132,
        23,  27,  33,  26,  38,   7,  31,  39,  13,  25])

In [43]:
# 같은 동네에 같은 규모의 동일 업종이 많은듯? -> 사장 나이도 비슷....;
df['bizBzcCd'].unique()

array([47, 46, 10, 42, 56, 96, 45, 49, 86, 71, 73, 41, 95, 85, 94, 29, 76,
       55, 68, 91, 30, 20, 72, 25, 33, 16, 32, 23, 90, 35, 26, 52, 14, 59,
       58, 13, 75, 18, 28, 17, 22, 27, 15, 63, 62, 74, 24, 38, 87, 34, 31,
       37,  1, 12,  3, 50])

In [47]:
tmp = df.groupby(['basYm', 'bizAreaNm', 'bizBzcCd']).count().reset_index()

In [49]:
tmp=tmp.iloc[:, :4]

In [54]:
tmp.columns = tmp.columns[:-1].tolist() + ['count']

In [60]:
tmp

,basYm,bizAreaNm,bizBzcCd,count
0,202208,강원도 강릉시,10,236
1,202208,강원도 강릉시,41,44
2,202208,강원도 강릉시,42,380
3,202208,강원도 강릉시,45,596
4,202208,강원도 강릉시,46,884
...,...,...,...,...
16378,202406,충청북도 충주시 앙성면,56,92
16379,202406,충청북도 충주시 앙성면,76,24
16380,202406,충청북도 충주시 앙성면,85,12
16381,202406,충청북도 충주시 앙성면,86,18


In [69]:
kind_changes = tmp.pivot_table(index=['bizAreaNm', 'bizBzcCd'], columns='basYm', values='count', aggfunc='sum').reset_index()

In [ ]:
# 지역별로 업종 변화 확인 가능.... 근데 다 없어진게 말이 되나...;

In [ ]:
df[['bizBzcCd', 'bizBzcCdNm']].drop_duplicates().sort_values(by='bizBzcCd').to_csv(f'{path}/dataset/업종분류.csv', index=False)

In [ ]:
kind = pd.read_csv(f'{path}/dataset/업종분류.csv')

In [71]:
kind_changes = kind_changes.merge(kind, on='bizBzcCd')

In [ ]:
kind_changes
# 3개년 데이터 다 있는 데이터들 확인 필요

,bizAreaNm,bizBzcCd,202208,202306,202406,bizBzcCdNm
0,강원도 강릉시,10,236.0,NaN,NaN,식료품 제조업
1,강원도 강릉시,41,44.0,NaN,NaN,종합 건설업
2,강원도 강릉시,42,380.0,NaN,NaN,전문직별 공사업
3,강원도 강릉시,45,596.0,NaN,NaN,자동차 및 부품 판매업
4,강원도 강릉시,46,884.0,NaN,NaN,도매 및 상품 중개업
...,...,...,...,...,...,...
10167,충청북도 충주시 앙성면,56,NaN,126.0,92.0,음식점 및 주점업
10168,충청북도 충주시 앙성면,76,NaN,26.0,24.0,임대업; 부동산 제외
10169,충청북도 충주시 앙성면,85,NaN,26.0,12.0,교육 서비스업
10170,충청북도 충주시 앙성면,86,NaN,28.0,18.0,보건업


In [ ]:
kind_changes[kind_changes['bizAreaNm'] == '강원도 강릉시'] # 강릉시는 2023이후로 값이 없음... -> 다른곳도 그런 곳 있는지 확인 필요

,bizAreaNm,bizBzcCd,202208,202306,202406,bizBzcCdNm
0,강원도 강릉시,10,236.0,NaN,NaN,식료품 제조업
1,강원도 강릉시,41,44.0,NaN,NaN,종합 건설업
2,강원도 강릉시,42,380.0,NaN,NaN,전문직별 공사업
3,강원도 강릉시,45,596.0,NaN,NaN,자동차 및 부품 판매업
4,강원도 강릉시,46,884.0,NaN,NaN,도매 및 상품 중개업
5,강원도 강릉시,47,944.0,NaN,NaN,소매업; 자동차 제외
6,강원도 강릉시,49,368.0,NaN,NaN,육상 운송 및 파이프라인 운송업
7,강원도 강릉시,56,1048.0,NaN,NaN,음식점 및 주점업
8,강원도 강릉시,71,20.0,NaN,NaN,전문 서비스업
9,강원도 강릉시,73,28.0,NaN,NaN,기타 전문 과학 및 기술 서비스업


In [ ]:
# 정보가 없는건지 폐업한건지 확인 필요...

In [ ]:
kind_changes[kind_changes[202406].isna()][['bizAreaNm', 'bizBzcCd', 'bizBzcCdNm']].drop_duplicates()  # 분석 불가??

,bizAreaNm,bizBzcCd,bizBzcCdNm
0,강원도 강릉시,10,식료품 제조업
1,강원도 강릉시,41,종합 건설업
2,강원도 강릉시,42,전문직별 공사업
3,강원도 강릉시,45,자동차 및 부품 판매업
4,강원도 강릉시,46,도매 및 상품 중개업
...,...,...,...
10144,충청북도 충주시,91,스포츠 및 오락관련 서비스업
10145,충청북도 충주시,95,개인 및 소비용품 수리업
10146,충청북도 충주시,96,기타 개인 서비스업
10150,충청북도 충주시 거룡2길,26,전자 부품 컴퓨터 영상 음향 및 통신장비 제조업
